# TABLAS RIPS 

## Carga de los datos

Este jupyter notebook esta generando las tablas sobre un año de la fuente de datos RIPS

In [1]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("GeneraciónTablasRips_Todoslosdatos2022").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/15 14:09:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Leer los archivos Parquet de un año 
df1_parquet = spark.read.parquet("/rawdata/rips/rips_parquet_2022")

df=df1_parquet

In [3]:
# Contar el número de registros en el DataFrame unido
total_registros = df.count()

# Listar las columnas del DataFrame unido
columnas = df.columns

# Mostrar resultados
print(f"Total de registros en el DataFrame unido: {total_registros}")
print(f"Columnas del DataFrame unido: {columnas}")

Total de registros en el DataFrame unido: 465644990
Columnas del DataFrame unido: ['personaid', 'tipoeventoripsdesc', 'codigo', 'dxprincipal', 'dxegreso', 'finalidadprocedimientoscd', 'finalidadconsultacd', 'tipousuariocd', 'causaexternacd', 'prestador', 'estadosalidacd', 'tipodiagnosticoprincipalcd', 'codigoprocedimiento', 'municipiocd', 'fechaid', 'edad', 'sexodesc', 'costoconsulta', 'costoprocedimiento', 'netoapagarconsulta', 'numerodiasestancia', '__null_dask_index__']


In [4]:
# Contar el número de registros - da satisfactoriamente la cantidad de registros de toda la fuente RIPS 4’262.860.903 registros 
record_count = df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo es: {record_count}")

El número de registros en el archivo es: 465644990


In [5]:
# Contar el número total de columnas en el DataFrame ( no se tiene en cuenta neto a pagar)
total_columns = len(df.columns)

# Imprimir el número total de columnas
print("Número total de columnas en el DataFrame:", total_columns)

Número total de columnas en el DataFrame: 22


In [6]:
# Mostrar el esquema (columnas y tipos de datos)
df.printSchema()

# Listar las columnas
print(df.columns)

root
 |-- personaid: string (nullable = true)
 |-- tipoeventoripsdesc: string (nullable = true)
 |-- codigo: string (nullable = true)
 |-- dxprincipal: string (nullable = true)
 |-- dxegreso: string (nullable = true)
 |-- finalidadprocedimientoscd: string (nullable = true)
 |-- finalidadconsultacd: string (nullable = true)
 |-- tipousuariocd: string (nullable = true)
 |-- causaexternacd: string (nullable = true)
 |-- prestador: string (nullable = true)
 |-- estadosalidacd: string (nullable = true)
 |-- tipodiagnosticoprincipalcd: string (nullable = true)
 |-- codigoprocedimiento: string (nullable = true)
 |-- municipiocd: string (nullable = true)
 |-- fechaid: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- sexodesc: string (nullable = true)
 |-- costoconsulta: string (nullable = true)
 |-- costoprocedimiento: string (nullable = true)
 |-- netoapagarconsulta: string (nullable = true)
 |-- numerodiasestancia: string (nullable = true)
 |-- __null_dask_index__: long (nul

# Empieza el proceso de generación de los datos agrupados por atenciones

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
spark = SparkSession.builder.appName("DataFrame agrupado").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df = df1

# Seleccionar solo las columnas deseadas (sin 'personaid_count' ya que no existe aún)
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd'
]
df_filtered = df.select(*columns_to_keep)

# Agregar una nueva columna con el recuento distintivo de 'personaid'
window_spec = Window.partitionBy('personaid')
df_filtered = df_filtered.withColumn('personaid_count', F.count('personaid').over(window_spec))

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered.show()

# Mostrar los nombres de las columnas filtradas para verificar
#print("Nombres de las columnas filtradas:")
#print(df_filtered.columns)

# Agrupar por la columna 'personaid'
grouped_df = df_filtered.groupBy('personaid').agg(
    F.first('codigoprocedimiento').alias('codigoprocedimiento'),
    F.first('tipoeventoripsdesc').alias('tipoeventoripsdesc'),
    F.first('fechaid').alias('fechaid'),
    F.first('sexodesc').alias('sexodesc'),
    F.first('tipodiagnosticoprincipalcd').alias('tipodiagnosticoprincipalcd'),
    F.first('edad').alias('edad'),
    F.first('dxegreso').alias('dxegreso'),
    F.first('dxprincipal').alias('dxprincipal'),
    F.first('municipiocd').alias('municipiocd'),
    F.first('personaid_count').alias('personaid_count')
)

# Mostrar las primeras filas del DataFrame agrupado para verificar
grouped_df.show()


24/08/15 14:10:10 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---------+-------------------+------------------+--------+--------+--------------------------+----+--------+-----------+-----------+---------------+
|personaid|codigoprocedimiento|tipoeventoripsdesc| fechaid|sexodesc|tipodiagnosticoprincipalcd|edad|dxegreso|dxprincipal|municipiocd|personaid_count|
+---------+-------------------+------------------+--------+--------+--------------------------+----+--------+-----------+-----------+---------------+
|100000046|             890105|         CONSULTAS|20220114|FEMENINO|                         1|  28|       0|       U072|      25430|             69|
|100000046|             890301|         CONSULTAS|20220304|FEMENINO|                         3|  28|       0|       U071|      25430|             69|
|100000046|             890301|         CONSULTAS|20220304|FEMENINO|                         3|  28|       0|       U071|      25430|             69|
|100000046|             890301|         CONSULTAS|20220304|FEMENINO|                         3|  28|

+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|personaid|codigoprocedimiento|  tipoeventoripsdesc| fechaid| sexodesc|tipodiagnosticoprincipalcd|edad|dxegreso|dxprincipal|municipiocd|personaid_count|
+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|100000046|             890105|           CONSULTAS|20220114| FEMENINO|                         1|  28|       0|       U072|      25430|             69|
|100001340|             890302|           CONSULTAS|20220511|MASCULINO|                         1|  28|       0|       Z719|      05001|              2|
|100002120|             890201|           CONSULTAS|20220428|MASCULINO|                         1|  33|       0|       R074|      15542|             11|
|100002546|             890239|           CONSULTAS|20220621| FEMENINO|           

In [8]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año da 73'357.494 llego hasta stage 18
#llega a 30'434.249 con 2022
record_count = grouped_df.count()

# Mostrar el número de registros
print(f"El número de registros en el dataframe es: {record_count}")

El número de registros en el dataframe es: 30434249


Se generan un total de 73'357.494 registros (todos) para el año 2022 son 30'434.249  registros

In [9]:
 df_final =grouped_df


In [10]:
# Contar el número de registros 
record_count = df_final.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 30434249


In [11]:
# Mostrar las primeras filas del DataFrame agrupado para verificar
df_final.show()

+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|personaid|codigoprocedimiento|  tipoeventoripsdesc| fechaid| sexodesc|tipodiagnosticoprincipalcd|edad|dxegreso|dxprincipal|municipiocd|personaid_count|
+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|100000046|             890308|           CONSULTAS|20220307| FEMENINO|                         2|  28|       0|       I10X|      25430|             69|
|100001340|             890302|           CONSULTAS|20220514|MASCULINO|                         1|  28|       0|       Z719|      05001|              2|
|100002120|             890376|           CONSULTAS|20221005|MASCULINO|                         1|  33|       0|       H261|      15542|             11|
|100002546|             890239|           CONSULTAS|20220621| FEMENINO|           

In [12]:
#se escribe el parquet - No lo hago por que seria todos los datos por atenciones, mejor solo de cada tabla
#df_final.write.parquet("hdfs:/analyticsdata/todoRips.parquet") 

# Empieza el proceso de generación de tablas

## 1. Agrupado por año y mes

In [13]:
# Cargar el archivo Parquet en un DataFrame
df2 = df_final

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
spark = SparkSession.builder.appName("DataFrameTabla1 ").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
df2 = df_final




24/08/12 21:57:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [15]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = df2.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 30434249


In [16]:
# Mostrar el esquema (columnas y tipos de datos)
df2.printSchema()

# Listar las columnas
print(df2.columns)

root
 |-- personaid: string (nullable = true)
 |-- codigoprocedimiento: string (nullable = true)
 |-- tipoeventoripsdesc: string (nullable = true)
 |-- fechaid: string (nullable = true)
 |-- sexodesc: string (nullable = true)
 |-- tipodiagnosticoprincipalcd: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- dxegreso: string (nullable = true)
 |-- dxprincipal: string (nullable = true)
 |-- municipiocd: string (nullable = true)
 |-- personaid_count: long (nullable = true)

['personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid', 'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count']


# resultado final agrupado año y mes, según el numero de personas y atenciones

In [17]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año y mes
result_df = df_filtered.groupBy('Año', 'Mes').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Filtrar para mostrar los resultados de enero para comprobar
result_january = result_df.filter((F.col('Año') == 2022) & (F.col('Mes') == 1))

# Mostrar los resultados para enero
result_january.show()

# Mostrar las primeras filas del DataFrame completo para verificar
result_df.show()

# Imprimir los nombres de las columnas agrupadas para verificar
#print("Nombres de las columnas agrupadas:")
#print(result_df.columns)

+----+---+------------------+--------------------+
| Año|Mes|Número_de_Personas|Número_de_Atenciones|
+----+---+------------------+--------------------+
|2022|  1|           2602650|            32999971|
+----+---+------------------+--------------------+



+----+----+------------------+--------------------+
| Año| Mes|Número_de_Personas|Número_de_Atenciones|
+----+----+------------------+--------------------+
|2022|  10|           2901565|            43700580|
|2022|   5|           3024483|            57882787|
|2022|   2|           1190051|            13511697|
|2022|  11|           2875831|            49857874|
|2022|   8|           2453734|            32611500|
|2022|   1|           2600502|            33012501|
|2022|   6|           3278042|            46728514|
|2022|   9|           2616721|            36843193|
|2022|   4|           2486222|            43092995|
|2022|  12|           2324485|            31662916|
|2022|   7|           2166954|            26093345|
|2022|   3|           2515657|            50647086|
|null|null|                 2|                   2|
+----+----+------------------+--------------------+



In [18]:
result_df.show()

+----+----+------------------+--------------------+
| Año| Mes|Número_de_Personas|Número_de_Atenciones|
+----+----+------------------+--------------------+
|2022|  10|           2908075|            44257067|
|2022|   5|           3031475|            57973013|
|2022|   2|           1194483|            13596006|
|2022|  11|           2892616|            50395996|
|2022|   8|           2435062|            32090944|
|2022|   1|           2612981|            33386532|
|2022|   6|           3263427|            46192044|
|2022|   9|           2605041|            36689280|
|2022|   4|           2503243|            43308468|
|2022|  12|           2303056|            31026660|
|2022|   7|           2143389|            25256303|
|2022|   3|           2541399|            51472675|
|null|null|                 2|                   2|
+----+----+------------------+--------------------+



Modificar el nombre del archivo segun corresponda el año

In [19]:
#se escribe el parquet tabla 1
result_df.write.parquet("hdfs:/analyticsdata/1.Tabla_anio_mes_atenciones_2022.parquet") 

# 2. Agrupado por año, mes y sexo

In [20]:

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y sexodesc
result_df = df_filtered.groupBy('Año', 'Mes', 'sexodesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+----+-----------+------------------+--------------------+
| Año| Mes|   sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+----+-----------+------------------+--------------------+
|2022|   7|   FEMENINO|           1182102|            15269472|
|2022|  10|       null|              6094|               84623|
|2022|   6|  MASCULINO|           1413284|            17271551|
|2022|   3|  MASCULINO|           1025305|            18562087|
|2022|   5|       null|              4417|               80308|
|2022|   2|       null|              2041|               25559|
|2022|  11|       null|              6213|               90900|
|2022|   2|   FEMENINO|            666411|             8334929|
|2022|   8|       null|              4563|               59642|
|2022|  11|   FEMENINO|           1657318|            31789630|
|2022|   9|  MASCULINO|           1145612|            14330909|
|2022|  12|   FEMENINO|           1260601|            18845824|
|2022|  12|NO DEFINIDO|              817

In [21]:
#se escribe el parquet tabla 2
result_df.write.parquet("hdfs:/analyticsdata/2.Tabla_anio_mes_sexo_2022.parquet") 

# 3. Agrupado por año, mes y municipio

In [14]:

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y municipioid
result_df = df_filtered.groupBy('Año', 'Mes', 'municipiocd').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+-----------+------------------+--------------------+
| Año|Mes|municipiocd|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+------------------+--------------------+
|2022|  4|      18592|              1378|               24370|
|2022| 11|      15109|               304|                5518|
|2022| 12|      15238|              6751|              107453|
|2022|  6|      08137|              1479|               19379|
|2022|  8|      76122|               948|               14073|
|2022|  2|      27001|              3280|               90354|
|2022|  4|      73555|              1237|               21555|
|2022| 12|      52317|               717|                4339|
|2022|  8|      20750|              1166|               23740|
|2022|  7|      76377|               371|                4825|
|2022|  5|      05036|                93|                1069|
|2022|  9|      52560|               383|                2582|
|2022|  5|      70204|               632|              

In [15]:
#se escribe el parquet tabla 3
result_df.write.parquet("hdfs:/analyticsdata/3.Tabla_anio_mes_municipio_2022.parquet") 

# 4. Agrupado por año, mes y tipoeventoripsdesc

In [22]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y tipoeventoripsdesc
result_df = df_filtered.groupBy('Año', 'Mes', 'tipoeventoripsdesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+----+--------------------+------------------+--------------------+
| Año| Mes|  tipoeventoripsdesc|Número_de_Personas|Número_de_Atenciones|
+----+----+--------------------+------------------+--------------------+
|2022|  11|           CONSULTAS|           1703624|            28122703|
|2022|   9|           URGENCIAS|             32990|              279179|
|2022|  12|           CONSULTAS|           1395130|            15663856|
|2022|   2|PROCEDIMIENTOS DE...|            411048|             4848138|
|2022|  10|   HOSPITALIZACIONES|             32072|             1542992|
|2022|   7|PROCEDIMIENTOS DE...|            638510|             8267199|
|2022|   2|           CONSULTAS|            752145|             8120055|
|2022|   1|           URGENCIAS|             32246|              606890|
|2022|  11|   HOSPITALIZACIONES|             26870|             1074555|
|2022|  10|PROCEDIMIENTOS DE...|           1045343|            16943510|
|2022|   8|           URGENCIAS|             25926|

In [23]:
#se escribe el parquet tabla 4
result_df.write.parquet("hdfs:/analyticsdata/4.Tabla_anio_mes_tipoeventoripsdesc_2022.parquet") 

# 5. Agrupado por año, mes y tipodiagnosticoprincipalcd

In [24]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y tipodiagnosticoprincipalcd
result_df = df_filtered.groupBy('Año', 'Mes', 'tipodiagnosticoprincipalcd').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+----+--------------------------+------------------+--------------------+
| Año| Mes|tipodiagnosticoprincipalcd|Número_de_Personas|Número_de_Atenciones|
+----+----+--------------------------+------------------+--------------------+
|2022|  10|                      null|           1121463|            19014514|
|2022|   9|                         1|            997201|            12143106|
|2022|   6|                         1|           1208731|            14103965|
|2022|   7|                         3|            173531|             2572894|
|2022|   2|                         2|            143641|             1547762|
|2022|   6|                         2|            395364|             4308136|
|2022|   7|                         2|            316631|             3318026|
|2022|   5|                      null|           1405849|            30349613|
|2022|   5|                         3|            208469|             4684097|
|2022|  10|                         3|            23

In [25]:
#se escribe el parquet tabla 5
result_df.write.parquet("hdfs:/analyticsdata/5.Tabla_anio_mes_tipodiagnosticoprincipalcd_2022.parquet") 

# 6. Agrupado por año, mes y dxprincipal

In [26]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y dxprincipal
result_df = df_filtered.groupBy('Año', 'Mes', 'dxprincipal').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+------------------+--------------------+
| Año|Mes|dxprincipal|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+------------------+--------------------+
|2022|  9|       N751|               178|                2127|
|2022|  2|       E669|              5273|               44289|
|2022| 10|       F718|                85|                1272|
|2022|  7|       S025|              2652|               30983|
|2022|  8|       F322|               378|                7541|
|2022| 10|       S008|               454|                3552|
|2022|  5|       K121|               221|                2408|
|2022| 12|       K046|               781|                9770|
|2022| 11|       K055|               457|                4971|
|2022|  5|       Y471|                13|                 224|
|2022|  3|       I959|               157|                7239|
|2022|  1|       F129|                17|                 443|
|2022|  5|       J950|                16|              

In [27]:
#se escribe el parquet tabla 6
result_df.write.parquet("hdfs:/analyticsdata/6.Tabla_anio_mes_dxprincipal_2022.parquet") 

# 7. Agrupado por año, mes y codigoprocedimiento

In [16]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'codigoprocedimiento').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-------------------+------------------+--------------------+
| Año|Mes|codigoprocedimiento|Número_de_Personas|Número_de_Atenciones|
+----+---+-------------------+------------------+--------------------+
|2022|  6|             903842|              1503|               29328|
|2022|  6|             881401|              6482|               82944|
|2022|  3|             873420|              2734|               32158|
|2022|  7|             865101|              3304|               15734|
|2022| 12|             890750|              1288|               24041|
|2022|  9|             890351|               707|               24068|
|2022|  8|             750302|               325|                1189|
|2022| 12|             662202|               944|                2352|
|2022| 10|             903883|              1817|               68664|
|2022|  8|             882317|              1296|               15016|
|2022| 10|             890438|               103|                3190|
|2022|

In [17]:
#se escribe el parquet tabla 7
result_df.write.parquet("hdfs:/analyticsdata/7.Tabla_anio_mes_codigoprocedimiento_2022.parquet") 

# 8. Agrupado por año, mes, municipio y sexo

In [35]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'municipiocd','sexodesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+-----------+------------------+--------------------+
| Año|Mes|municipiocd|   sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+-----------+------------------+--------------------+
|2022|  9|      41668|   FEMENINO|                10|                  14|
|2022|  7|      70678|   FEMENINO|                 6|                  10|
|2022| 10|      47189|   FEMENINO|               618|                1553|
|2022|  8|      25307|   FEMENINO|                39|                  52|
|2022| 11|      25394|  MASCULINO|                 4|                  16|
|2022|  4|      23466|   FEMENINO|                 4|                   5|
|2022|  1|      15407|   FEMENINO|                 6|                   7|
|2022|  9|      76736|  MASCULINO|                 2|                   2|
|2022|  7|       null|   FEMENINO|                36|                  44|
|2022|  2|      25269|   FEMENINO|               399|                1428|
|2022|  9|      70713|  M

In [29]:
#se escribe el parquet tabla 8
result_df.write.parquet("hdfs:/analyticsdata/8.Tabla_anio_mes_municipio_sexo_2022.parquet") 

# 9. Agrupado por año, mes, tipoeventorips y sexo

In [31]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'tipoeventoripsdesc','sexodesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+----+--------------------+-----------+------------------+--------------------+
| Año| Mes|  tipoeventoripsdesc|   sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+----+--------------------+-----------+------------------+--------------------+
|2022|   7|   HOSPITALIZACIONES|  MASCULINO|             11517|              405221|
|2022|   6|           CONSULTAS|NO DEFINIDO|              6615|               51071|
|2022|  11|           CONSULTAS|       null|              4045|               48297|
|2022|   1|           CONSULTAS|  MASCULINO|            695204|             8081806|
|2022|   9|           URGENCIAS|       null|                88|                 535|
|2022|  12|           CONSULTAS|       null|              4236|               34118|
|2022|   2|PROCEDIMIENTOS DE...|       null|               620|                8730|
|2022|  10|   HOSPITALIZACIONES|       null|                83|                5203|
|2022|   1|PROCEDIMIENTOS DE...|   FEMENINO|            517719|  

In [32]:
#se escribe el parquet tabla 9
result_df.write.parquet("hdfs:/analyticsdata/9.Tabla_anio_mes_tipoeventoripsdesc_sexo_2022.parquet") 

In [ ]:
se comprueba que el archivo quedo guardado correctamente al leerlo 

In [33]:
# Leer el archivo .parquet y se guarda en un DataFrame
df_leer = spark.read.parquet("hdfs:/analyticsdata/9.Tabla_anio_mes_tipoeventoripsdesc_sexo_2022.parquet")

# Mostrar el contenido del DataFrame
df_leer.show()

+----+---+--------------------+-----------+------------------+--------------------+
| Año|Mes|  tipoeventoripsdesc|   sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------------+-----------+------------------+--------------------+
|2022|  8|           URGENCIAS|   FEMENINO|             12244|              103189|
|2022|  8|PROCEDIMIENTOS DE...|  MASCULINO|            353181|             5564769|
|2022|  6|   HOSPITALIZACIONES|  MASCULINO|             10281|              352742|
|2022|  5|           URGENCIAS|   FEMENINO|              9848|               83089|
|2022| 10|           URGENCIAS|NO DEFINIDO|               184|                1450|
|2022| 11|   HOSPITALIZACIONES|   FEMENINO|             14663|              629901|
|2022|  7|           CONSULTAS|       null|              3111|               33166|
|2022|  2|   HOSPITALIZACIONES|  MASCULINO|              3159|               71579|
|2022| 10|           URGENCIAS|   FEMENINO|             17292|              

# 10. Agrupado por año, mes, edad

In [10]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'edad').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+----+------------------+--------------------+
| Año|Mes|edad|Número_de_Personas|Número_de_Atenciones|
+----+---+----+------------------+--------------------+
|2022|  6|  55|             35994|              617919|
|2022|  1|  49|             28865|              389481|
|2022|  8|  22|             38818|              425761|
|2022|  6|  67|             26480|              615298|
|2022| 11|  16|             39292|              487635|
|2022| 11|  45|             33855|              542792|
|2022|  8|   4|             39047|              396237|
|2022|  9|   1|             36882|              561110|
|2022|  5|  66|             28817|              851936|
|2022| 10|  25|             45043|              571466|
|2022|  8|  26|             40102|              433152|
|2022| 10|  79|             11725|              355560|
|2022|  2|  29|             19923|              189579|
|2022| 11|  56|             35277|              723299|
|2022|  6|   6|             49437|              

In [12]:
#se escribe el parquet tabla 10
result_df.write.parquet("hdfs:/analyticsdata/10.Tabla_anio_mes_edad_2022.parquet") 

In [18]:
spark.stop()